In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기

In [2]:
train = pd.read_csv('C:/dacon/train.csv')
test = pd.read_csv('C:/dacon/test.csv')
submission = pd.read_csv('C:/dacon/sample_submission.csv')
car_2020 = pd.read_csv('C:/dacon/car_2020.csv',encoding='cp949')

# 전용면적당 법정주차대수 

## 법정주차대수 Column 추가

In [3]:
train.insert(loc=14, column='법정주차대수', value=0.0)

## 전용면적을 지역별 각 지역 기준으로 나누기

In [4]:
# for문 columns 설정
Gwang_colName = ['부산광역시', '대전광역시', '광주광역시', '울산광역시', '대구광역시', '세종광역시', '경기도']
Other_colName = ['경상북도', '경상남도', '전라북도', '전라남도', '강원도', '충청남도', '충청북도']

# 1. 서울특별시 
# - 전용면적 {85이하 - 75} | {85초과 - 65} 나누기
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 85) ].index
train.loc[idx, '법정주차대수'] = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 85) ]['전용면적'] / 75

idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 85) ].index
train.loc[idx, '법정주차대수'] = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 85) ]['전용면적'] / 65

# 2. 광역시 + 경기도
# - 전용면적 {85이하 - 85} | {85초과 - 70} 나누기
for colName in Gwang_colName:
    idx = train[ (train['지역'] == colName) & (train['전용면적'] <= 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] <= 85) ]['전용면적'] / 85
    idx = train[(train['지역'] == colName) & (train['전용면적'] > 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] > 85) ]['전용면적'] / 70
    
# 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 전용면적 {85이하 - 95} | {85초과 - 75} 나누기
for colName in Other_colName:
    idx = train[ (train['지역'] == colName) & (train['전용면적'] <= 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] <= 85) ]['전용면적'] / 95
    idx = train[(train['지역'] == colName) & (train['전용면적'] > 85) ].index
    train.loc[idx, '법정주차대수'] = train[(train['지역'] == colName) & (train['전용면적'] > 85) ]['전용면적'] / 75

## 법정주차대수 소수점 정리

In [5]:
# 1. 서울특별시 
# 1) 전용면적 30이하 : 0.5 고정
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] <= 30) ].index
train.loc[idx, '법정주차대수'] = 0.5

# 2) 전용면적 30초과 ~ 60이하 : 0.8 고정
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 30) & (train['전용면적'] <= 60) ].index
train.loc[idx, '법정주차대수'] = 0.8

# 3) 전용면적 60초과 : 1.xxx -> 2 or 2.xxxx -> 3
# - 나머지가 조금이라도 나오면 몫의 값 + 1 => np.ceil 함수 사용
idx = train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 60) ].index
train.loc[idx, '법정주차대수'] = np.ceil(train[ (train['지역'] == '서울특별시') & (train['전용면적'] > 60) ]['법정주차대수'])

# 2. 광역시 + 경기도
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
for colName in Gwang_colName:
    idx = train[ train['지역'] == colName ].index
    train.loc[idx, '법정주차대수'] = np.ceil(train[ train['지역'] == colName ]['법정주차대수'])

# 3. 강원도, 충청남도 등 (1과 2 제외한 나머지 지역 OO도)
# - 필요한 조건은 앞서 한 조건(나누기 작업)만 하면 되기 때문에 소수점 올림만 작업하면 된다.
for colName in Other_colName:
    idx = train[ train['지역'] == colName ].index
    train.loc[idx, '법정주차대수'] = np.ceil(train[ train['지역'] == colName ]['법정주차대수'])

# 아파트 / 상가 데이터 분리

In [6]:
apart_train = train[train['임대건물구분'] == '아파트']
shop_train = train[train['임대건물구분'] == '상가']

# 컬럼명 바꾸기

In [6]:
train = train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

test = test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

# 결측치 처리

In [8]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       569
임대료         569
지하철         211
버스            4
단지내주차면수       0
법정주차대수        0
등록차량수         0
dtype: int64
-------- test.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64


In [7]:
# train 데이터
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)
train['지하철'] = train['지하철'].fillna(0)
train['버스'] = train['버스'].fillna(0)

# test 데이터
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

In [10]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
법정주차대수      0
등록차량수       0
dtype: int64
-------- test.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64


# '-' 데이터 처리

In [8]:
# 임대료, 임대보증금 데이터에 - 값이 있어서 이를 0으로 변환
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)

In [26]:
a=pd.DataFrame(train.corrwith(train['등록차량수']).abs().sort_values(ascending=False))

In [27]:
train_1 = train.copy()
train_1  = train_1.drop_duplicates()
b=pd.DataFrame(train_1.corrwith(train_1['등록차량수']).abs().sort_values(ascending=False))

In [36]:
pd.concat([a,b],axis=1,join='inner')

,0,0
등록차량수,1.000000,1.000000
단지내주차면수,0.859216,0.851280
임대료,0.519834,0.461294
임대보증금,0.436073,0.378680
총세대수,0.316703,0.423656
전용면적별세대수,0.247085,0.188906
공가수,0.118121,0.095454
전용면적,0.115453,0.081118
버스,0.102293,0.117849
지하철,0.086562,0.063716


# 지하철 Y or N로 변경

In [12]:
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)
test['지하철'] = np.where(test['지하철'] == 0, 0, 1)

# 임대건물구분 컬럼 처리

In [13]:
# 임대건물구분에서 7개의 아파트는 상가로 예상이됨 이를 처리 함.
# 임대건물구분이 상가인 경우는 전용면적별세대수도 1이므로 이를 처리 함.

idx = train[(train['임대건물구분']=='아파트') & (train['자격유형']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

# 지역 데이터를 숫자로 매핑

In [14]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

# 전용면적을 상/하한 적용

In [15]:
# 전용면적을 5의 배수로 변경 후 상/하한 적용
# 하한 값 15, 상한 값 100을 적용

train['전용면적'] = np.round(train['전용면적'], 0)
train['전용면적'] = train['전용면적']//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

test['전용면적'] = np.round(test['전용면적'], 0)
test['전용면적'] = test['전용면적']//5*5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

# 학습 데이터 생성

In [16]:
train_data = train[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수','등록차량수']]
test_data = test[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수']]
print("train 데이터:",train_data.shape)
print("test 데이터:",test_data.shape)

train 데이터: (2952, 8)
test 데이터: (1022, 7)


In [17]:
# 데이터 target 설정

x_train = train_data.iloc[:,:7]
y_train = train_data.iloc[:,-1]
x_test = test_data
print("x_train 데이터:",x_train.shape)
print("y_train 데이터:",y_train.shape)
print("x_train 데이터:",x_test.shape)

x_train 데이터: (2952, 7)
y_train 데이터: (2952,)
x_train 데이터: (1022, 7)


# 모델링

In [18]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_jobs=-1, random_state=42)

forest.fit(x_train, y_train)

pred = forest.predict(x_test)

In [19]:
import xgboost

xgb = xgboost.XGBRegressor()

xgb.fit(x_train, y_train)

pred = xgb.predict(x_test)

# 예측 데이터셋 생성

In [20]:
df = []
df = pd.DataFrame(df)
df['code'] = test['단지코드']
df['num'] = pred
sub = df.groupby(['code'],sort=False,as_index=False).mean()

In [21]:
# sub.to_csv('xgb.csv', index=False)
sub.head()

,code,num
0,C1072,649.207397
1,C1128,1191.849976
2,C1456,638.613586
3,C1840,467.848450
4,C1332,1009.499634


# 모델 평가

In [22]:
from sklearn.model_selection import train_test_split

x = train_data.iloc[:,:7]
y = train_data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0)

from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(random_state=0)

forest.fit(x_train, y_train)

pred = forest.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("mae :",mean_absolute_error(y_test, pred))

from sklearn.metrics import r2_score
print("r2_score :",r2_score(y_test, pred))

mae : 28.853726287262873
r2_score : 0.9783652648951725


In [23]:
from sklearn.model_selection import train_test_split

x = train_data.iloc[:,:7]
y = train_data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0)

import xgboost

xgb = xgboost.XGBRegressor(random_state=0)

xgb.fit(x_train, y_train)

pred = xgb.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("mae :",mean_absolute_error(y_test, pred))

from sklearn.metrics import r2_score
print("r2_score :",r2_score(y_test, pred))

mae : 29.864864125807433
r2_score : 0.9804879974711456
